In [1]:
import pandas as pd
import numpy as np

In [2]:
### Get all the pillar names from the excel

In [3]:
names = pd.read_excel('../../UNDP Digital Assessment Data Framework Filename Matching V7.xlsx')

In [4]:
col_names = ['Indicator','check', 'Data Source','Data Link','Index','Filename','Sub-Pillar']

In [5]:
names = names[col_names]

In [6]:
names.head()

,Indicator,check,Data Source,Data Link,Index,Filename,Sub-Pillar
0,Countries,NaN,UN Statistics Division: List of Countries,https://unstats.un.org,False,Countries,NaN
1,"Database of Global Administrative Areas (GADM,...",NaN,NaN,https://gadm.org,False,NaN,NaN
2,High Resolution Population Density Maps + Demo...,NaN,NaN,NaN,False,NaN,NaN
3,population density vs openstreetmap object den...,NaN,NaN,NaN,False,NaN,NaN
4,Population Density,Infrastructure,World Bank: World Development Indicators,https://datacatalog.worldbank.org,False,population_density,Connectivity technology


In [7]:
# get all the files per pillar
data_stats = names.groupby('check').agg({'Filename':'count','Indicator':'count'})

In [8]:
data_stats

,Filename,Indicator
check,,
Business,18,27
Foundations,13,22
Government,10,15
Infrastructure,47,58
People,35,49
Regulation,5,8
Strategy,1,1


In [9]:
### People

In [10]:
bnames = names[(names.check=='People')&(~names.Filename.isna())]#&(names.Index==False)]
bnames

,Indicator,check,Data Source,Data Link,Index,Filename,Sub-Pillar
113,Human Capital Index (HCI),People,UN: E-Government Survey,https://publicadministration.un.org/egovkb,True,e_government_index,Digital Literacy Skills
114,% of population using internet (all),People,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,False,ITU_database,Digital Literacy Skills
115,% of population using internet (female),People,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,False,ITU_database,Digital Literacy Skills
116,% of population using internet (male),People,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,False,ITU_database,Digital Literacy Skills
117,% of internet users who own cryptocurrency,People,Statista,https://www.statista.com,False,cryptocurrency_adoption,Digital Literacy Skills
118,Global Crypto Adoption Index,People,Chainanalysis: Global Crypto Adoption Index,https://go.chainalysis.com,TRUE,Chainalysis_2021_Geography_Cryptocurrency_Report,Usage and Ownership
119,SDG 4.4 Digital literacy data,People,UNESCO: Sustainable Development Goals,http://data.uis.unesco.org,False,SDG_digital_literacy_data,Digital Literacy Skills
120,UNDP Human Development Index (HDI),People,UNDP: Human Development Data Center,http://hdr.undp.org/en,True,undp_human_developmnt,Culture
121,Facebook Social Connectedness Index,People,Facebook: Social Connectedness Index,https://data.humdata.org,True,fb_social_connectedness,Culture
122,Share of individuals using the Internet to int...,People,OECD: Going Digital Toolkit,http://goingdigital.oecd.org,False,population_interacting_public_officials,Culture


In [11]:
# get list of names for all indicators
indicators = bnames.Indicator.unique()
subpillars = bnames['Sub-Pillar'].unique()

In [12]:
# get all file names
bfiles = bnames.Filename.unique()

In [13]:
bfiles

array(['e_government_index', 'ITU_database', 'cryptocurrency_adoption',
       'Chainalysis_2021_Geography_Cryptocurrency_Report',
       'SDG_digital_literacy_data', 'undp_human_developmnt',
       'fb_social_connectedness',
       'population_interacting_public_officials',
       'digital_public_service_use', 'apps_in_national_language',
       'time_spent_online', 'happiness_score',
       'not_buying_online_concern_about_returning',
       'not_buying_online_concern_about_security',
       'ewaste_per_inhabitant', 'automation_led_unemployment',
       'cyberbullying_rate', 'global_wellbeing_initiative',
       'financial_inclusiveness', 'individuals_buying_online_frequency',
       'social_media_penetration', 'FB_users', 'gender_gaps',
       'population_digital_financial_services',
       'tax_percent_mobile_ownership', 'population_with_smartphones'],
      dtype=object)

In [14]:
subpillars

array(['Digital Literacy Skills', 'Usage and Ownership', 'Culture',
       'Digital wellbeing'], dtype=object)

In [15]:
# formula for converting scale 0-100
def convert_rank(old_value, old_min=0, old_max=100, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [16]:
# formula for converting scale 0-1
def convert_rank_b(old_value, old_min=0, old_max=1, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [17]:
# formula for converting scale 0-10
def convert_rank_c(old_value, old_min=0, old_max=10, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [18]:
### 1. Human Capital Index (HCI)

In [19]:
indicators[0]

# load data
indicator = indicators[0]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Human Capital Index (HCI)
e_government_index


In [20]:
df.head(10)

,Survey Year,Country Name,E-Government Rank,E-Government Index,E-Participation Index,Online Service Index,Human Capital Index,Telecommunication Infrastructure Index
0,2020,Iraq,143,0.4360,0.3095,0.3353,0.4358,0.5370
1,2020,Ireland,27,0.8433,0.8571,0.7706,0.9494,0.8100
2,2020,Israel,30,0.8361,0.7143,0.7471,0.8924,0.8689
3,2020,Italy,37,0.8231,0.8214,0.8294,0.8466,0.7932
4,2020,Jamaica,114,0.5392,0.3690,0.3882,0.7142,0.5151
5,2020,Japan,14,0.8989,0.9881,0.9059,0.8684,0.9223
6,2020,Jordan,117,0.5309,0.3333,0.3588,0.6800,0.5540
7,2020,Kazakhstan,29,0.8375,0.8810,0.9235,0.8866,0.7024
8,2020,Kenya,116,0.5326,0.5952,0.6765,0.5812,0.3402
9,2020,Kiribati,145,0.4320,0.5595,0.4941,0.6778,0.1241


In [21]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [22]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Human Capital Index'] 
df['Year'] = df['Survey Year']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_b(row,old_min=0,old_max=1))

In [23]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Iraq,2020,Human Capital Index (HCI),0.4358,3.179,True,Digital Literacy Skills
1,Ireland,2020,Human Capital Index (HCI),0.9494,5.747,True,Digital Literacy Skills
2,Israel,2020,Human Capital Index (HCI),0.8924,5.462,True,Digital Literacy Skills
3,Italy,2020,Human Capital Index (HCI),0.8466,5.233,True,Digital Literacy Skills
4,Jamaica,2020,Human Capital Index (HCI),0.7142,4.571,True,Digital Literacy Skills
...,...,...,...,...,...,...,...
188,Senegal,2020,Human Capital Index (HCI),0.3332,2.666,True,Digital Literacy Skills
189,Serbia,2020,Human Capital Index (HCI),0.8280,5.140,True,Digital Literacy Skills
190,Seychelles,2020,Human Capital Index (HCI),0.7660,4.830,True,Digital Literacy Skills
191,Singapore,2020,Human Capital Index (HCI),0.8904,5.452,True,Digital Literacy Skills


In [24]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [25]:
### 2. % of population using internet (all)

In [26]:
indicators[1]

# load data
indicator = indicators[1]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population using internet (all)
ITU_database


In [27]:
df.head(150)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
145,North Macedonia,Europe,MKD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
146,Norway,Europe,NOR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
147,Poland,Europe,POL,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
148,Portugal,Europe,PRT,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [28]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [29]:
df = df[(df.Year==2019)]
df = df[(df.iloc[:,3] =='Individuals using the Internet, total (%)')]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [30]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
47607,Cabo Verde,2019.0,% of population using internet (all),61.943398,4.097170,True,Digital Literacy Skills
47608,Central African Rep.,2019.0,% of population using internet (all),NaN,NaN,True,Digital Literacy Skills
47609,Congo (Rep. of the),2019.0,% of population using internet (all),NaN,NaN,True,Digital Literacy Skills
47610,Côte d'Ivoire,2019.0,% of population using internet (all),36.288955,2.814448,True,Digital Literacy Skills
47611,Equatorial Guinea,2019.0,% of population using internet (all),NaN,NaN,True,Digital Literacy Skills
...,...,...,...,...,...,...,...
47739,Saint Vincent and the Grenadines,2019.0,% of population using internet (all),NaN,NaN,True,Digital Literacy Skills
47740,Trinidad and Tobago,2019.0,% of population using internet (all),NaN,NaN,True,Digital Literacy Skills
47741,United States,2019.0,% of population using internet (all),89.430285,5.471514,True,Digital Literacy Skills
47742,Uruguay,2019.0,% of population using internet (all),83.351534,5.167577,True,Digital Literacy Skills


In [31]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [32]:
### 3. % of population using internet (female)

In [33]:
indicators[2]

# load data
indicator = indicators[2]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population using internet (female)
ITU_database


In [34]:
df.head(10)

# Must convert the string in the dataset to float

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [35]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [36]:
df = df[(df.iloc[:,3] =='Female Internet users as a % of total female population')]
df = df[(df.Year==2019)]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value']
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [37]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
40307,Cabo Verde,2019.0,% of population using internet (female),60.965017,4.048251,True,Digital Literacy Skills
40308,Côte d'Ivoire,2019.0,% of population using internet (female),32.924445,2.646222,True,Digital Literacy Skills
40309,Kenya,2019.0,% of population using internet (female),20.125122,2.006256,True,Digital Literacy Skills
40310,Lesotho,2019.0,% of population using internet (female),44.922768,3.246138,True,Digital Literacy Skills
40311,Mauritius,2019.0,% of population using internet (female),60.130645,4.006532,True,Digital Literacy Skills
...,...,...,...,...,...,...,...
40388,Mexico,2019.0,% of population using internet (female),68.574653,4.428733,True,Digital Literacy Skills
40389,Panama,2019.0,% of population using internet (female),63.811494,4.190575,True,Digital Literacy Skills
40390,Paraguay,2019.0,% of population using internet (female),69.017230,4.450861,True,Digital Literacy Skills
40391,Peru,2019.0,% of population using internet (female),56.992144,3.849607,True,Digital Literacy Skills


In [38]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [39]:
### 4. % of population using internet (male)

In [40]:
indicators[3]

# load data
indicator = indicators[3]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population using internet (male)
ITU_database


In [41]:
df.head(10)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [42]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [43]:
df = df[(df.iloc[:,3] =='Male Internet users as a % of total male population')]
df = df[(df.Year==2019)]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value']
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [44]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
41078,Cabo Verde,2019.0,% of population using internet (male),62.921166,4.146058,True,Digital Literacy Skills
41079,Côte d'Ivoire,2019.0,% of population using internet (male),39.808871,2.990444,True,Digital Literacy Skills
41080,Kenya,2019.0,% of population using internet (male),25.071668,2.253583,True,Digital Literacy Skills
41081,Lesotho,2019.0,% of population using internet (male),38.349985,2.917499,True,Digital Literacy Skills
41082,Mauritius,2019.0,% of population using internet (male),63.393093,4.169655,True,Digital Literacy Skills
...,...,...,...,...,...,...,...
41159,Mexico,2019.0,% of population using internet (male),71.740324,4.587016,True,Digital Literacy Skills
41160,Panama,2019.0,% of population using internet (male),63.433835,4.171692,True,Digital Literacy Skills
41161,Paraguay,2019.0,% of population using internet (male),68.006259,4.400313,True,Digital Literacy Skills
41162,Peru,2019.0,% of population using internet (male),62.915703,4.145785,True,Digital Literacy Skills


In [45]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [46]:
### 5. % of internet users who own cryptocurrency

In [47]:
indicators[4]

# load data
indicator = indicators[4]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

# The indicators are all jumbled, need to unclutter this

% of internet users who own cryptocurrency
cryptocurrency_adoption


In [48]:
df.head(15)

,Country,Share of respondents who indicated they either owned or used cryptocurrencies in 55 countries worldwide in 2020,Units
0,Nigeria,31.9,in %
1,Vietnam,21.1,in %
2,Philippines,19.8,in %
3,South Africa,17.8,in %
4,Thailand,17.6,in %
5,Peru,16.1,in %
6,Turkey,16.1,in %
7,Colombia,15.3,in %
8,Argentina,14.4,in %
9,Indonesia,13.0,in %


In [49]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [50]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Share of respondents who indicated they either owned or used cryptocurrencies in 55 countries worldwide in 2020'] 
df['Country Name'] = df['Country']
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [51]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Nigeria,2020,% of internet users who own cryptocurrency,31.9,2.595,True,Digital Literacy Skills
1,Vietnam,2020,% of internet users who own cryptocurrency,21.1,2.055,True,Digital Literacy Skills
2,Philippines,2020,% of internet users who own cryptocurrency,19.8,1.990,True,Digital Literacy Skills
3,South Africa,2020,% of internet users who own cryptocurrency,17.8,1.890,True,Digital Literacy Skills
4,Thailand,2020,% of internet users who own cryptocurrency,17.6,1.880,True,Digital Literacy Skills
5,Peru,2020,% of internet users who own cryptocurrency,16.1,1.805,True,Digital Literacy Skills
6,Turkey,2020,% of internet users who own cryptocurrency,16.1,1.805,True,Digital Literacy Skills
7,Colombia,2020,% of internet users who own cryptocurrency,15.3,1.765,True,Digital Literacy Skills
8,Argentina,2020,% of internet users who own cryptocurrency,14.4,1.720,True,Digital Literacy Skills
9,Indonesia,2020,% of internet users who own cryptocurrency,13.0,1.650,True,Digital Literacy Skills


In [52]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [53]:
### 6. Crypto Adoption Index

In [54]:
indicators[5]

# load data
indicator = indicators[5]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Global Crypto Adoption Index
Chainalysis_2021_Geography_Cryptocurrency_Report


In [55]:
df.head(15)

,Country,Score,Rank,On-chain value received (Rank),On-chain retail value received (Rank),P2P exchange trade volume (Rank)
0,Vietnam,1.00,1,4,2,3
1,India,0.37,2,2,3,72
2,Pakistan,0.36,3,11,12,8
3,Ukraine,0.29,4,6,5,40
4,Kenya,0.28,5,41,28,1
5,Nigeria,0.26,6,15,10,18
6,Venezuela,0.25,7,29,22,6
7,United States,0.22,8,3,4,109
8,Togo,0.19,9,47,42,2
9,Argentina,0.19,10,14,17,33


In [56]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [57]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Score'] 
df['Country Name'] = df['Country']
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_b(row,old_min=0,old_max=1))

In [58]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Vietnam,2019,Global Crypto Adoption Index,1.00,6.00,True,Usage and Ownership
1,India,2019,Global Crypto Adoption Index,0.37,2.85,True,Usage and Ownership
2,Pakistan,2019,Global Crypto Adoption Index,0.36,2.80,True,Usage and Ownership
3,Ukraine,2019,Global Crypto Adoption Index,0.29,2.45,True,Usage and Ownership
4,Kenya,2019,Global Crypto Adoption Index,0.28,2.40,True,Usage and Ownership
...,...,...,...,...,...,...,...
152,Guyana,2019,Global Crypto Adoption Index,0.00,1.00,True,Usage and Ownership
153,"Virgin Islands, U.S.",2019,Global Crypto Adoption Index,0.00,1.00,True,Usage and Ownership
154,Brunei Darussalam,2019,Global Crypto Adoption Index,0.00,1.00,True,Usage and Ownership
155,Bermuda,2019,Global Crypto Adoption Index,0.00,1.00,True,Usage and Ownership


In [59]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [60]:
### 7. SDG 4.4 Digital literacy data

In [61]:
indicators[6]

# load data
indicator = indicators[6]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))


SDG 4.4 Digital literacy data
SDG_digital_literacy_data


In [62]:
df.head(15)

,SDG_IND,Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags
0,ICTSKILLTRANSFERFILE_M,Proportion of youth and adults who have transf...,BRA,Brazil,2014,2014,23.31007,NaN,NaN
1,ICTSKILLTRANSFERFILE_M,Proportion of youth and adults who have transf...,BRA,Brazil,2016,2016,21.53173,NaN,NaN
2,ICTSKILLTRANSFERFILE_M,Proportion of youth and adults who have transf...,BRA,Brazil,2017,2017,21.14880,NaN,NaN
3,ICTSKILLTRANSFERFILE_M,Proportion of youth and adults who have transf...,BRA,Brazil,2018,2018,21.84886,NaN,NaN
4,ICTSKILLDUPLIC_M,Proportion of youth and adults who have used c...,BRA,Brazil,2014,2014,25.64427,NaN,NaN
5,ICTSKILLDUPLIC_M,Proportion of youth and adults who have used c...,BRA,Brazil,2016,2016,22.01463,NaN,NaN
6,ICTSKILLDUPLIC_M,Proportion of youth and adults who have used c...,BRA,Brazil,2017,2017,22.69577,NaN,NaN
7,ICTSKILLDUPLIC_M,Proportion of youth and adults who have used c...,BRA,Brazil,2018,2018,22.59234,NaN,NaN
8,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,TUR,Turkey,2014,2014,27.72807,NaN,NaN
9,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,TUR,Turkey,2015,2015,25.66142,NaN,NaN


In [63]:
df = df[(df.Time == 2019)]
df['Country Name'] = df['Country']
df.head(15)

,SDG_IND,Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags,Country Name
12,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,TUR,Turkey,2019,2019,38.18440,NaN,NaN,Turkey
35,ICTSKILLSOFTWARE,"Proportion of youth and adults who have found,...",THA,Thailand,2019,2019,5.30000,NaN,NaN,Thailand
75,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,PAK,Pakistan,2019,2019,2.70000,NaN,NaN,Pakistan
90,ICTSKILLPROGLANG,Proportion of youth and adults who have wrote ...,PRT,Portugal,2019,2019,8.20000,NaN,NaN,Portugal
103,ICTSKILLSOFTWARE,"Proportion of youth and adults who have found,...",FIN,Finland,2019,2019,63.10000,NaN,NaN,Finland
121,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,FRA,France,2019,2019,54.60000,NaN,NaN,France
126,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,ESP,Spain,2019,2019,54.67742,NaN,NaN,Spain
133,ICTSKILLATTACH,Proportion of youth and adults who have sent e...,KAZ,Kazakhstan,2019,2019,51.70000,NaN,NaN,Kazakhstan
143,ICTSKILLPROGLANG,Proportion of youth and adults who have wrote ...,KAZ,Kazakhstan,2019,2019,6.30000,NaN,NaN,Kazakhstan
152,ICTSKILLPROGLANG,Proportion of youth and adults who have wrote ...,BEL,Belgium,2019,2019,4.50000,NaN,NaN,Belgium


In [64]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [65]:
# Since there can be several entries for the same country, calculate the mean of the value for each country
agg_df = df.groupby(['Country Name']).agg({'Value':'mean'})
agg_df.columns = ['Mean' ]
agg_df

,Mean
Country Name,
Albania,11.528571
Austria,48.320000
Bahrain,47.700000
Bangladesh,0.600000
Belarus,26.811111
...,...
Thailand,11.944444
Tunisia,18.888889
Turkey,28.284636


In [66]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
agg_df['higher_is_better'] = True
agg_df['Indicator'] = indicator
agg_df['data_col'] = agg_df['Mean']
agg_df['Year'] = 2019
agg_df['Sub-Pillar'] = subpillar

min_rank = agg_df['data_col'].min()
max_rank = agg_df['data_col'].max()

# transform 0-1 rank into 1-6
agg_df['new_rank_score'] = agg_df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [67]:
agg_df = agg_df[['Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
agg_df

,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
Country Name,,,,,,
Albania,2019,SDG 4.4 Digital literacy data,11.528571,1.937960,True,Digital Literacy Skills
Austria,2019,SDG 4.4 Digital literacy data,48.320000,5.095635,True,Digital Literacy Skills
Bahrain,2019,SDG 4.4 Digital literacy data,47.700000,5.042423,True,Digital Literacy Skills
Bangladesh,2019,SDG 4.4 Digital literacy data,0.600000,1.000000,True,Digital Literacy Skills
Belarus,2019,SDG 4.4 Digital literacy data,26.811111,3.249605,True,Digital Literacy Skills
...,...,...,...,...,...,...
Thailand,2019,SDG 4.4 Digital literacy data,11.944444,1.973653,True,Digital Literacy Skills
Tunisia,2019,SDG 4.4 Digital literacy data,18.888889,2.569669,True,Digital Literacy Skills
Turkey,2019,SDG 4.4 Digital literacy data,28.284636,3.376072,True,Digital Literacy Skills


In [68]:
### 8. UNDP Human Development Index (HDI)

In [69]:
indicators[7]

# load data
indicator = indicators[7]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

UNDP Human Development Index (HDI) 
undp_human_developmnt


In [70]:
df.head()

,HDI rank,Country,Value,Unnamed: 3,(years),(2017 PPP $),data_country,data_year
0,NaN,NaN,2019,NaN,2019,2019,NaN,NaN
1,NaN,VERY HIGH HUMAN DEVELOPMENT,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,Norway,0.957,NaN,12.89775,66494.25217,NaN,NaN
3,2.0,Ireland,0.955,NaN,12.6663305,68370.58737,NaN,NaN
4,2.0,Switzerland,0.955,NaN,13.38081241,69393.52076,NaN,NaN


In [71]:
# choose only the rows where column HDI rank is numeric

df = df.iloc[0:194,:]
df = df[pd.to_numeric(df['HDI rank'], errors='coerce').notnull()]

In [72]:
df.head(67)

,HDI rank,Country,Value,Unnamed: 3,(years),(2017 PPP $),data_country,data_year
2,1.0,Norway,0.957,NaN,12.89775,66494.25217,NaN,NaN
3,2.0,Ireland,0.955,NaN,12.6663305,68370.58737,NaN,NaN
4,2.0,Switzerland,0.955,NaN,13.38081241,69393.52076,NaN,NaN
5,4.0,"Hong Kong, China (SAR)",0.949,NaN,12.27996,62984.76553,NaN,NaN
6,4.0,Iceland,0.949,NaN,12.77278684,54682.38057,NaN,NaN
...,...,...,...,...,...,...,...,...
64,62.0,Malaysia,0.81,NaN,10.37283,27534.09856,NaN,NaN
65,64.0,Kuwait,0.806,NaN,7.275667996,58590.08219,NaN,NaN
66,64.0,Serbia,0.806,NaN,11.19411,17191.66873,NaN,NaN
67,66.0,Mauritius,0.804,NaN,9.54,25266.21195,NaN,NaN


In [73]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [74]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'].astype(float)
df['Country Name'] = df['Country']
df['Year'] = 2019
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_b(row,old_min=0,old_max=1))

In [75]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
2,Norway,2019,UNDP Human Development Index (HDI),0.957,5.785,True,Usage and Ownership
3,Ireland,2019,UNDP Human Development Index (HDI),0.955,5.775,True,Usage and Ownership
4,Switzerland,2019,UNDP Human Development Index (HDI),0.955,5.775,True,Usage and Ownership
5,"Hong Kong, China (SAR)",2019,UNDP Human Development Index (HDI),0.949,5.745,True,Usage and Ownership
6,Iceland,2019,UNDP Human Development Index (HDI),0.949,5.745,True,Usage and Ownership
...,...,...,...,...,...,...,...
189,Burundi,2019,UNDP Human Development Index (HDI),0.433,3.165,True,Usage and Ownership
190,South Sudan,2019,UNDP Human Development Index (HDI),0.433,3.165,True,Usage and Ownership
191,Chad,2019,UNDP Human Development Index (HDI),0.398,2.990,True,Usage and Ownership
192,Central African Republic,2019,UNDP Human Development Index (HDI),0.397,2.985,True,Usage and Ownership


In [76]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator, index=False))

In [77]:
### 9. Facebook Social Connectedness Index

In [78]:
indicators[8]

# load data
indicator = indicators[8]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Facebook Social Connectedness Index
fb_social_connectedness


In [79]:
df.head(25)

,user_loc,fr_loc,scaled_sci
0,1001,AE,8729
1,1001,AG,95256
2,1001,AL,3122
3,1001,AM,3470
4,1001,AO,2839
5,1001,AR,3729
6,1001,AT,6977
7,1001,AU,21136
8,1001,AW,27607
9,1001,AZ,1108


In [80]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Culture


In [81]:
df['Country Name'] = df['fr_loc']

In [82]:
agg_df = df.groupby(['Country Name']).agg({'scaled_sci':'mean'})
agg_df

,scaled_sci
Country Name,
AE,10370.571075
AG,184463.177454
AL,8414.715701
AM,2825.352741
AO,3777.043357
...,...
XK,9482.109322
YT,1927.744503
ZA,9507.912666


In [83]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
agg_df['higher_is_better'] = True
agg_df['Indicator'] = indicator
agg_df['data_col'] = agg_df['scaled_sci'] 
agg_df['Sub-Pillar'] = subpillar
agg_df['Year'] = 2021

min_rank = agg_df['data_col'].min()
max_rank = agg_df['data_col'].max()

# transform 0-1 rank into 1-6
agg_df['new_rank_score'] = agg_df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [84]:
agg_df = agg_df[['Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
agg_df

# Need to find a way to convert ISO codes to full country names

,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
Country Name,,,,,,
AE,2021,Facebook Social Connectedness Index,10370.571075,1.011298,True,Culture
AG,2021,Facebook Social Connectedness Index,184463.177454,1.231631,True,Culture
AL,2021,Facebook Social Connectedness Index,8414.715701,1.008823,True,Culture
AM,2021,Facebook Social Connectedness Index,2825.352741,1.001749,True,Culture
AO,2021,Facebook Social Connectedness Index,3777.043357,1.002953,True,Culture
...,...,...,...,...,...,...
XK,2021,Facebook Social Connectedness Index,9482.109322,1.010174,True,Culture
YT,2021,Facebook Social Connectedness Index,1927.744503,1.000613,True,Culture
ZA,2021,Facebook Social Connectedness Index,9507.912666,1.010206,True,Culture


In [85]:
### 10. Share of individuals using the Internet to interact with officials

In [86]:
indicators[9]

# load data
indicator = indicators[9]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Share of individuals using the Internet to interact with public authorities
population_interacting_public_officials


In [87]:
df.head(15)

,Country,Indicator,Breakdowns,Time,Unit,PowerCode,Reference Period,Value,Flags
0,Australia,Individuals using the Internet for downloading...,All (individuals aged 16-74),2010,Percentage,Units,NaN,38.1100,Difference in methodology
1,Australia,Individuals using the Internet for downloading...,All (individuals aged 16-74),2012,Percentage,Units,NaN,49.9600,Difference in methodology
2,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2005,Percentage,Units,NaN,29.1940,NaN
3,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2006,Percentage,Units,NaN,32.9733,NaN
4,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2007,Percentage,Units,NaN,27.4741,NaN
5,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2008,Percentage,Units,NaN,51.2252,Break
6,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2009,Percentage,Units,NaN,48.8515,NaN
7,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2010,Percentage,Units,NaN,51.0458,NaN
8,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2011,Percentage,Units,NaN,51.2893,NaN
9,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2012,Percentage,Units,NaN,52.8274,NaN


In [88]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Culture


In [89]:
# filter most recent year
df = df[(df.Time==2019)]
df = df[(df.Indicator=='Individuals using the Internet for visiting or interacting with public authorities websites - last 12 m (%)')]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Year'] = df['Time']
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [90]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
16,Austria,2019,Share of individuals using the Internet to int...,69.735500,4.486775,True,Culture
64,Belgium,2019,Share of individuals using the Internet to int...,58.605000,3.930250,True,Culture
109,Brazil,2019,Share of individuals using the Internet to int...,34.194129,2.709706,True,Culture
148,Colombia,2019,Share of individuals using the Internet to int...,6.274581,1.313729,True,Culture
169,Czech Republic,2019,Share of individuals using the Internet to int...,53.776600,3.688830,True,Culture
216,Denmark,2019,Share of individuals using the Internet to int...,91.670900,5.583545,True,Culture
262,Estonia,2019,Share of individuals using the Internet to int...,80.015600,5.000780,True,Culture
310,Finland,2019,Share of individuals using the Internet to int...,87.295400,5.364770,True,Culture
357,France,2019,Share of individuals using the Internet to int...,74.724200,4.736210,True,Culture
397,Germany,2019,Share of individuals using the Internet to int...,59.097700,3.954885,True,Culture


In [91]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [92]:
### 11. Level of satisfaction for online public service

In [93]:
indicators[10]

# load data
indicator = indicators[10]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Level of satisfaction for online public services (% of users, by type of interaction and service)
digital_public_service_use


In [94]:
df.head(10)

,Country Name,User Satisfaction (%)
0,United Arab Emirates,88.5
1,India,86.1
2,Saudi Arabia,84.2
3,China,81.8
4,Singapore,80.9
5,France,79.1
6,Chile,78.3
7,Hong Kong,76.6
8,Kenya,76.1
9,Poland,75.2


In [95]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Culture


In [96]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['User Satisfaction (%)'] 
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

# Need to replace the % mark 

In [97]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,United Arab Emirates,2020,Level of satisfaction for online public servic...,88.5,5.425,True,Culture
1,India,2020,Level of satisfaction for online public servic...,86.1,5.305,True,Culture
2,Saudi Arabia,2020,Level of satisfaction for online public servic...,84.2,5.210,True,Culture
3,China,2020,Level of satisfaction for online public servic...,81.8,5.090,True,Culture
4,Singapore,2020,Level of satisfaction for online public servic...,80.9,5.045,True,Culture
5,France,2020,Level of satisfaction for online public servic...,79.1,4.955,True,Culture
6,Chile,2020,Level of satisfaction for online public servic...,78.3,4.915,True,Culture
7,Hong Kong,2020,Level of satisfaction for online public servic...,76.6,4.830,True,Culture
8,Kenya,2020,Level of satisfaction for online public servic...,76.1,4.805,True,Culture
9,Poland,2020,Level of satisfaction for online public servic...,75.2,4.760,True,Culture


In [98]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [99]:
### 12. Number of mobile apps available in national language

In [100]:
indicators[11]

# load data
indicator = indicators[11]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Number of mobile apps available in national language(s)
apps_in_national_language


In [101]:
df.head(15)

,Unnamed: 0,ISO Code,Country,Region,Year,Number of apps in national language
0,1,AFG,Afghanistan,South Asia,2014,2.444741
1,2,AFG,Afghanistan,South Asia,2015,2.793221
2,3,AFG,Afghanistan,South Asia,2016,2.849881
3,4,AFG,Afghanistan,South Asia,2017,2.913741
4,5,AFG,Afghanistan,South Asia,2018,2.961247
5,6,AFG,Afghanistan,South Asia,2019,3.000000
6,7,AGO,Angola,Sub-Saharan Africa,2014,53.333237
7,8,AGO,Angola,Sub-Saharan Africa,2015,55.080910
8,9,AGO,Angola,Sub-Saharan Africa,2016,56.516411
9,10,AGO,Angola,Sub-Saharan Africa,2017,57.061077


In [102]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Culture


In [103]:
# filter most recent year and global value
df = df[(df.Year==2019)]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Number of apps in national language'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [104]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,Number of mobile apps available in national la...,3.000000,1.150000,True,Culture
11,Angola,2019,Number of mobile apps available in national la...,57.978500,3.898925,True,Culture
17,Albania,2019,Number of mobile apps available in national la...,67.286446,4.364322,True,Culture
23,United Arab Emirates,2019,Number of mobile apps available in national la...,78.453438,4.922672,True,Culture
29,Argentina,2019,Number of mobile apps available in national la...,89.724289,5.486214,True,Culture
...,...,...,...,...,...,...,...
995,Samoa,2019,Number of mobile apps available in national la...,58.999996,3.950000,True,Culture
1001,Yemen,2019,Number of mobile apps available in national la...,78.453438,4.922672,True,Culture
1007,South Africa,2019,Number of mobile apps available in national la...,57.271473,3.863574,True,Culture
1013,Zambia,2019,Number of mobile apps available in national la...,16.000000,1.800000,True,Culture


In [105]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [106]:
### 13. Device Addiction (time of use on internet)

In [107]:
indicators[12]

# load data
indicator = indicators[12]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Device Addiction (time of use on internet / on devices)
time_spent_online


In [108]:
df.head(15)

# Need to move the row further down

,Country,"Average daily time spent using the internet by online users worldwide as of 3rd quarter 2020, by region (in hours.minutes)"
0,Philippines,10.56
1,Brazil,10.08
2,Colombia,10.07
3,South Africa,10.06
4,Argentina,9.39
5,Malaysia,9.17
6,Mexico,9.01
7,Indonesia,8.52
8,Thailand,8.44
9,Taiwan,8.08


In [109]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital wellbeing


In [110]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Average daily time spent using the internet by online users worldwide as of 3rd quarter 2020, by region (in hours.minutes)'] 
df['Country Name'] = df['Country']
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [111]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Philippines,2020,Device Addiction (time of use on internet / on...,10.56,6.000000,True,Digital wellbeing
1,Brazil,2020,Device Addiction (time of use on internet / on...,10.08,5.619651,True,Digital wellbeing
2,Colombia,2020,Device Addiction (time of use on internet / on...,10.07,5.611727,True,Digital wellbeing
3,South Africa,2020,Device Addiction (time of use on internet / on...,10.06,5.603803,True,Digital wellbeing
4,Argentina,2020,Device Addiction (time of use on internet / on...,9.39,5.072900,True,Digital wellbeing
5,Malaysia,2020,Device Addiction (time of use on internet / on...,9.17,4.898574,True,Digital wellbeing
6,Mexico,2020,Device Addiction (time of use on internet / on...,9.01,4.771791,True,Digital wellbeing
7,Indonesia,2020,Device Addiction (time of use on internet / on...,8.52,4.383518,True,Digital wellbeing
8,Thailand,2020,Device Addiction (time of use on internet / on...,8.44,4.320127,True,Digital wellbeing
9,Taiwan,2020,Device Addiction (time of use on internet / on...,8.08,4.034865,True,Digital wellbeing


In [112]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(bf), index=False)

In [113]:
### 14. Gross National Wellbeing

In [114]:
indicators[13]

# load data
indicator = indicators[13]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

# happiness_score is missing looks like the problem is in a redundant space before it in the filename matching

Gross National Wellbeing
happiness_score


In [115]:
df.head(15)

,RANK,COUNTRY/ECONOMY,VALUE,SCORE
0,1.0,Finland,7.78,100.00
1,2.0,Switzerland,7.69,98.31
2,3.0,Denmark,7.69,98.28
3,4.0,Iceland,7.53,95.13
4,5.0,Norway,7.44,93.35
5,6.0,Netherlands,7.43,93.02
6,7.0,Luxembourg,7.40,92.60
7,8.0,Sweden,7.40,92.49
8,9.0,Ireland,7.25,89.67
9,10.0,Australia,7.23,89.26


In [116]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital wellbeing


In [117]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['SCORE'] 
df['Country Name'] = df['COUNTRY/ECONOMY']
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [118]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Finland,2019,Gross National Wellbeing,100.00,6.0000,True,Digital wellbeing
1,Switzerland,2019,Gross National Wellbeing,98.31,5.9155,True,Digital wellbeing
2,Denmark,2019,Gross National Wellbeing,98.28,5.9140,True,Digital wellbeing
3,Iceland,2019,Gross National Wellbeing,95.13,5.7565,True,Digital wellbeing
4,Norway,2019,Gross National Wellbeing,93.35,5.6675,True,Digital wellbeing
...,...,...,...,...,...,...,...
129,Zambia,2019,Gross National Wellbeing,12.06,1.6030,True,Digital wellbeing
130,Rwanda,2019,Gross National Wellbeing,11.30,1.5650,True,Digital wellbeing
131,India,2019,Gross National Wellbeing,10.92,1.5460,True,Digital wellbeing
132,Zimbabwe,2019,Gross National Wellbeing,0.00,1.0000,True,Digital wellbeing


In [119]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [120]:
### 15. Percentage of individuals not buying online due to concerns about returning products

In [121]:
indicators[14]

# load data
indicator = indicators[14]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Percentage of individuals not buying online due to concerns about returning products
not_buying_online_concern_about_returning


In [122]:
df.head(15)

,Indicator,Country,Variable,Unit,Scope,Time,Value,Flags
0,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2005,10.358200,NaN
1,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2006,14.697300,NaN
2,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2009,25.081000,NaN
3,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2015,17.049300,NaN
4,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2017,15.114670,NaN
5,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2019,16.727490,NaN
6,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2005,7.911900,NaN
7,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2006,13.751800,NaN
8,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2009,24.068800,NaN
9,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2015,12.671000,NaN


In [123]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital wellbeing


In [124]:
# filter most recent year and global value
df = df[(df.Time==2019)]
df = df[(df.Scope =='All individuals (aged 16-74)')]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Year'] = df['Time']
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (6-row)+1)

In [125]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Austria,2019,Percentage of individuals not buying online du...,16.727490,5.163626,False,Digital wellbeing
22,Belgium,2019,Percentage of individuals not buying online du...,9.164280,5.541786,False,Digital wellbeing
37,Czech Republic,2019,Percentage of individuals not buying online du...,8.857784,5.557111,False,Digital wellbeing
51,Denmark,2019,Percentage of individuals not buying online du...,13.273220,5.336339,False,Digital wellbeing
67,Estonia,2019,Percentage of individuals not buying online du...,3.968984,5.801551,False,Digital wellbeing
121,European Union (28 countries),2019,Percentage of individuals not buying online du...,17.256510,5.137175,False,Digital wellbeing
135,Finland,2019,Percentage of individuals not buying online du...,46.960710,3.651965,False,Digital wellbeing
152,France,2019,Percentage of individuals not buying online du...,14.236370,5.288182,False,Digital wellbeing
167,Germany,2019,Percentage of individuals not buying online du...,16.450070,5.177497,False,Digital wellbeing
184,Greece,2019,Percentage of individuals not buying online du...,9.599324,5.520034,False,Digital wellbeing


In [126]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [127]:
### 16. Percentage of individuals not buying online due to payment security concerns

In [128]:
indicators[15]

# load data
indicator = indicators[15]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Percentage of individuals not buying online due to payment security concerns
not_buying_online_concern_about_security


In [129]:
df.head(15)

,Indicator,Country,Variable,Unit,Scope,Time,Value,Flags
0,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2009,38.80930,NaN
1,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2015,34.89850,NaN
2,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2017,33.04743,NaN
3,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2019,31.19888,NaN
4,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2009,38.34010,NaN
5,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2015,34.96870,NaN
6,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2017,16.65675,NaN
7,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2019,12.88121,NaN
8,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 55-74,2009,40.57610,NaN
9,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 55-74,2015,42.60200,NaN


In [130]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital wellbeing


In [131]:
# filter most recent year and global value
df = df[(df.Time==2019)]
df = df[(df.Scope =='All individuals (aged 16-74)')]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Year'] = df['Time']
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (6-row)+1)

In [132]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
3,Austria,2019,Percentage of individuals not buying online du...,31.198880,4.440056,False,Digital wellbeing
15,Belgium,2019,Percentage of individuals not buying online du...,18.129610,5.093520,False,Digital wellbeing
27,Czech Republic,2019,Percentage of individuals not buying online du...,9.814267,5.509287,False,Digital wellbeing
39,Denmark,2019,Percentage of individuals not buying online du...,18.770550,5.061472,False,Digital wellbeing
51,Estonia,2019,Percentage of individuals not buying online du...,4.357608,5.782120,False,Digital wellbeing
84,European Union (28 countries),2019,Percentage of individuals not buying online du...,24.308600,4.784570,False,Digital wellbeing
96,Finland,2019,Percentage of individuals not buying online du...,64.427210,2.778639,False,Digital wellbeing
108,France,2019,Percentage of individuals not buying online du...,40.843490,3.957825,False,Digital wellbeing
120,Germany,2019,Percentage of individuals not buying online du...,25.121840,4.743908,False,Digital wellbeing
132,Greece,2019,Percentage of individuals not buying online du...,23.223810,4.838809,False,Digital wellbeing


In [133]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [134]:
### 17. E-waste generated, kilograms per inhabitant

In [135]:
indicators[16]

# load data
indicator = indicators[16]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

E-waste generated, kilograms per inhabitant
ewaste_per_inhabitant


In [136]:
df.head(15)

,iso3c,region_id,country_name,income_id,gdp,composition_food_organic_waste_percent,composition_glass_percent,composition_metal_percent,composition_other_percent,composition_paper_cardboard_percent,...,waste_treatment_controlled_landfill_percent,waste_treatment_incineration_percent,waste_treatment_landfill_unspecified_percent,waste_treatment_open_dump_percent,waste_treatment_other_percent,waste_treatment_recycling_percent,waste_treatment_sanitary_landfill_landfill_gas_system_percent,waste_treatment_unaccounted_for_percent,waste_treatment_waterways_marine_percent,where_where_is_this_data_measured
0,ABW,LCN,Aruba,HIC,35563.312500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11.000,NaN,89.00,NaN,NaN
1,AFG,SAS,Afghanistan,LIC,2057.062256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
2,AGO,SSF,Angola,LMC,8036.690430,51.80,6.70,4.40,11.50,11.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALB,ECS,Albania,UMC,13724.058594,51.40,4.50,4.80,15.21,9.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Some disposal sites
4,AND,ECS,Andorra,HIC,43711.800781,31.20,8.20,2.60,11.60,35.10,...,NaN,52.10,NaN,NaN,NaN,NaN,NaN,47.90,NaN,NaN
5,ARE,MEA,United Arab Emirates,HIC,67119.132812,39.00,4.00,3.00,10.00,25.00,...,NaN,NaN,9.00,62.0,NaN,20.000,NaN,NaN,NaN,NaN
6,ARG,LCN,Argentina,HIC,23550.099609,38.74,3.16,1.84,15.36,13.96,...,8.90,NaN,NaN,22.6,NaN,6.000,62.5,NaN,NaN,Other
7,ARM,ECS,Armenia,UMC,11019.838867,57.00,3.20,3.40,17.40,6.70,...,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,Other
8,ASM,EAS,American Samoa,UMC,11113.442383,19.70,3.40,7.90,25.60,26.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ATG,LCN,Antigua and Barbuda,HIC,17965.501953,46.00,7.00,7.00,12.00,15.00,...,98.68,NaN,NaN,NaN,NaN,NaN,NaN,1.14,0.1,Disposal Site


In [137]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital wellbeing


In [138]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
# Use special_waste_e_waste_tons_year times one thousand and divde by total population
df['data_col'] = df['special_waste_e_waste_tons_year']*1000/df['population_population_number_of_people'] 
df['Year'] = 2021
df['Country Name'] = df['country_name']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=min_rank,old_max=max_rank))

In [139]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Aruba,2021,"E-waste generated, kilograms per inhabitant",NaN,NaN,True,Digital wellbeing
1,Afghanistan,2021,"E-waste generated, kilograms per inhabitant",0.577100,1.040883,True,Digital wellbeing
2,Angola,2021,"E-waste generated, kilograms per inhabitant",3.665901,1.261006,True,Digital wellbeing
3,Albania,2021,"E-waste generated, kilograms per inhabitant",7.007240,1.499126,True,Digital wellbeing
4,Andorra,2021,"E-waste generated, kilograms per inhabitant",NaN,NaN,True,Digital wellbeing
...,...,...,...,...,...,...,...
212,Kosovo,2021,"E-waste generated, kilograms per inhabitant",0.012765,1.000666,True,Digital wellbeing
213,"Yemen, Rep.",2021,"E-waste generated, kilograms per inhabitant",1.522610,1.108265,True,Digital wellbeing
214,South Africa,2021,"E-waste generated, kilograms per inhabitant",6.205375,1.441981,True,Digital wellbeing
215,Zambia,2021,"E-waste generated, kilograms per inhabitant",1.051543,1.074694,True,Digital wellbeing


In [140]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [141]:
### 18. Automation-led unemployment

In [142]:
indicators[17]

# load data
indicator = indicators[17]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Automation-led unemployment
automation_led_unemployment


In [143]:
df.head(15)

,Country,No. of employees potentially automable (millions),Total employees (millions),Potential Rate of Automation (%)
0,Japan,35.60,63.90,55.71
1,Thailand,21.00,38.40,54.69
2,Senegal,2.20,4.07,54.00
3,Colombia,9.30,17.50,53.14
4,Peru,6.90,13.00,53.08
5,Taiwan,5.20,9.80,53.06
6,Kenya,7.40,14.20,52.11
7,South Korea,12.50,24.00,52.08
8,Sweden,2.10,4.04,52.00
9,Costa Rica,1.10,2.12,52.00


In [144]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital wellbeing


In [145]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
df['data_col'] = df['Potential Rate of Automation (%)'] 
df['Year'] = 2018
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (6-row)+1)

In [146]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Japan,2018,Automation-led unemployment,55.71,3.2145,False,Digital wellbeing
1,Thailand,2018,Automation-led unemployment,54.69,3.2655,False,Digital wellbeing
2,Senegal,2018,Automation-led unemployment,54.00,3.3000,False,Digital wellbeing
3,Colombia,2018,Automation-led unemployment,53.14,3.3430,False,Digital wellbeing
4,Peru,2018,Automation-led unemployment,53.08,3.3460,False,Digital wellbeing
5,Taiwan,2018,Automation-led unemployment,53.06,3.3470,False,Digital wellbeing
6,Kenya,2018,Automation-led unemployment,52.11,3.3945,False,Digital wellbeing
7,South Korea,2018,Automation-led unemployment,52.08,3.3960,False,Digital wellbeing
8,Sweden,2018,Automation-led unemployment,52.00,3.4000,False,Digital wellbeing
9,Costa Rica,2018,Automation-led unemployment,52.00,3.4000,False,Digital wellbeing


In [147]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [148]:
### 19. Cyberbullying

In [149]:
indicators[18]

# load data
indicator = indicators[18]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Cyberbullying
cyberbullying_rate


In [150]:
df.head(15)

# Need to move the top row down further

,2011,2016,2018,Country
0,32,32,37,India
1,20,19,29,Brazil
2,15,34,26,United States
3,12,13,25,Belgium
4,10,25,26,South Africa
5,--,--,23,Malaysia
6,14,20,23,Sweden
7,18,17,20,Canada
8,5,14,20,Turkey
9,18,17,19,Saudi Arabia


In [151]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital wellbeing


In [152]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['2018'] 
df['Year'] = 2018
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (6-row)+1)

In [153]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,India,2018,Cyberbullying,37,4.15,True,Digital wellbeing
1,Brazil,2018,Cyberbullying,29,4.55,True,Digital wellbeing
2,United States,2018,Cyberbullying,26,4.70,True,Digital wellbeing
3,Belgium,2018,Cyberbullying,25,4.75,True,Digital wellbeing
4,South Africa,2018,Cyberbullying,26,4.70,True,Digital wellbeing
5,Malaysia,2018,Cyberbullying,23,4.85,True,Digital wellbeing
6,Sweden,2018,Cyberbullying,23,4.85,True,Digital wellbeing
7,Canada,2018,Cyberbullying,20,5.00,True,Digital wellbeing
8,Turkey,2018,Cyberbullying,20,5.00,True,Digital wellbeing
9,Saudi Arabia,2018,Cyberbullying,19,5.05,True,Digital wellbeing


In [154]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [155]:
### 20. Global Wellbeing Initiative

In [156]:
indicators[19]

# load data
indicator = indicators[19]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

# Strange the global_wellbeing_initiative is clearly in the data manifest and the link works well 
# but it has not been moved to the processed folder

Global Wellbeing Initiative (World Happiness Index)
global_wellbeing_initiative


In [157]:
df.head(15)

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,...,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual,data_country,data_year
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.000,0.949,...,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253,NaN,NaN
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.700,0.946,...,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868,NaN,NaN
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.400,0.919,...,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839,NaN,NaN
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.000,0.955,...,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967,NaN,NaN
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.400,0.913,...,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798,NaN,NaN
5,Norway,Western Europe,7.392,0.035,7.462,7.323,11.053,0.954,73.300,0.960,...,2.43,1.543,1.108,0.782,0.703,0.249,0.427,2.580,NaN,NaN
6,Sweden,Western Europe,7.363,0.036,7.433,7.293,10.867,0.934,72.700,0.945,...,2.43,1.478,1.062,0.763,0.685,0.244,0.448,2.683,NaN,NaN
7,Luxembourg,Western Europe,7.324,0.037,7.396,7.252,11.647,0.908,72.600,0.907,...,2.43,1.751,1.003,0.760,0.639,0.166,0.353,2.653,NaN,NaN
8,New Zealand,North America and ANZ,7.277,0.040,7.355,7.198,10.643,0.948,73.400,0.929,...,2.43,1.400,1.094,0.785,0.665,0.276,0.445,2.612,NaN,NaN
9,Austria,Western Europe,7.268,0.036,7.337,7.198,10.906,0.934,73.300,0.908,...,2.43,1.492,1.062,0.782,0.640,0.215,0.292,2.784,NaN,NaN


In [158]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital wellbeing


In [159]:
# create standard columns
df.rename(columns={'Country name':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Ladder score'] 
df['Year'] = 2021
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()


df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_c(row, old_min=0,old_max=10))


In [160]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Finland,2021,Global Wellbeing Initiative (World Happiness I...,7.842,4.9210,True,Digital wellbeing
1,Denmark,2021,Global Wellbeing Initiative (World Happiness I...,7.620,4.8100,True,Digital wellbeing
2,Switzerland,2021,Global Wellbeing Initiative (World Happiness I...,7.571,4.7855,True,Digital wellbeing
3,Iceland,2021,Global Wellbeing Initiative (World Happiness I...,7.554,4.7770,True,Digital wellbeing
4,Netherlands,2021,Global Wellbeing Initiative (World Happiness I...,7.464,4.7320,True,Digital wellbeing
...,...,...,...,...,...,...,...
144,Lesotho,2021,Global Wellbeing Initiative (World Happiness I...,3.512,2.7560,True,Digital wellbeing
145,Botswana,2021,Global Wellbeing Initiative (World Happiness I...,3.467,2.7335,True,Digital wellbeing
146,Rwanda,2021,Global Wellbeing Initiative (World Happiness I...,3.415,2.7075,True,Digital wellbeing
147,Zimbabwe,2021,Global Wellbeing Initiative (World Happiness I...,3.145,2.5725,True,Digital wellbeing


In [161]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [162]:
### 21. Financial Inclusiveness

In [163]:
indicators[20]

# load data
indicator = indicators[20]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)


df = pd.read_csv('../../processed/{}.csv'.format(bf))
# Strange the global_wellbeing_initiative is clearly in the data manifest and the link works well 
# but it has not been moved to the processed folder

Financial Inclusiveness
financial_inclusiveness


In [164]:
df.head(15)

,Year,ISO,Country Name,Region,Income Group,Account (% age 15+),"Account, male (% age 15+)","Account, in labor force (% age 15+)","Account, out of labor force (% age 15+)","Account, female (% age 15+)",...,"Mobile money account, female (% age 15+)","Mobile money account, young adults (% age 15-24)","Mobile money account, older adults (% age 25+)","Mobile money account, primary education or less (% age 15+)","Mobile money account, secondary education or less (% age 15+)","Mobile money account, income, poorest 40% (% age 15+)","Mobile money account, income, richest 60% (% age 15+)","Mobile money account, rural (% age 15+)",data_country,data_year
0,2011,AFG,Afghanistan,South Asia,Low income,9%,15%,15%,2%,3%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,AFG,Afghanistan,South Asia,Low income,10%,16%,15%,4%,4%,...,0%,0%,0%,0%,0%,0%,1%,0%,NaN,NaN
2,2017,AFG,Afghanistan,South Asia,Low income,15%,23%,25%,4%,7%,...,1%,0%,1%,0%,2%,0%,1%,1%,NaN,NaN
3,2011,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,39%,39%,46%,31%,39%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,29%,36%,36%,12%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,28%,34%,36%,15%,23%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2014,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,38%,43%,44%,29%,34%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,40%,42%,53%,27%,38%,...,2%,6%,1%,1%,4%,0%,4%,2%,NaN,NaN
8,2011,ARB,Arab world,NaN,NaN,22%,30%,33%,11%,14%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014,ARB,Arab world,NaN,NaN,30%,38%,42%,18%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [166]:
df = df[(df.Year == 2017)]

# Must convert the data to float by removing the % sign
df['Account (% age 15+)'] = df['Account (% age 15+)'].str.replace('%','')
df['Account (% age 15+)'] = df['Account (% age 15+)'].astype(float)

In [167]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Account (% age 15+)'] 
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [168]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
2,Afghanistan,2017,Financial Inclusiveness,15.0,1.75,True,Usage and Ownership
7,Albania,2017,Financial Inclusiveness,40.0,3.00,True,Usage and Ownership
10,Arab world,2017,Financial Inclusiveness,37.0,2.85,True,Usage and Ownership
13,United Arab Emirates,2017,Financial Inclusiveness,88.0,5.40,True,Usage and Ownership
16,Argentina,2017,Financial Inclusiveness,49.0,3.45,True,Usage and Ownership
...,...,...,...,...,...,...,...
479,World,2017,Financial Inclusiveness,69.0,4.45,True,Usage and Ownership
482,Kosovo,2017,Financial Inclusiveness,52.0,3.60,True,Usage and Ownership
487,South Africa,2017,Financial Inclusiveness,69.0,4.45,True,Usage and Ownership
490,Zambia,2017,Financial Inclusiveness,46.0,3.30,True,Usage and Ownership


In [169]:
# output scores
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [170]:
### 22. E-commerce activity (% of individuals buying online and frequency)

In [171]:
indicators[21]

# load data
indicator = indicators[21]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

# Added _ between spaces to see if this changes anything

E-commerce activity (% of individuals buying online and frequency)
individuals_buying_online_frequency


In [172]:
df.head(15)

# Move the top row down further

,RANK,COUNTRY/ECONOMY,VALUE (%),SCORE,Year
0,1.0,Denmark,77.97,100.00,2017
1,2.0,Netherlands,75.72,97.11,2017
2,3.0,Norway,75.63,96.99,2017
3,4.0,United Kingdom,74.74,95.86,2017
4,5.0,"Korea, Rep.",72.47,92.94,2017
5,6.0,Sweden,71.65,91.90,2017
6,7.0,United States,70.43,90.33,2017
7,8.0,New Zealand,69.11,88.63,2017
8,9.0,Canada,68.57,87.95,2017
9,10.0,Australia,67.69,86.81,2017


In [173]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [174]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['VALUE (%)'] 
df['Country Name'] = df['COUNTRY/ECONOMY']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=min_rank,old_max=max_rank))

In [175]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Denmark,2017,E-commerce activity (% of individuals buying o...,77.97,6.000000,True,Usage and Ownership
1,Netherlands,2017,E-commerce activity (% of individuals buying o...,75.72,5.855658,True,Usage and Ownership
2,Norway,2017,E-commerce activity (% of individuals buying o...,75.63,5.849885,True,Usage and Ownership
3,United Kingdom,2017,E-commerce activity (% of individuals buying o...,74.74,5.792789,True,Usage and Ownership
4,"Korea, Rep.",2017,E-commerce activity (% of individuals buying o...,72.47,5.647164,True,Usage and Ownership
...,...,...,...,...,...,...,...
129,Iceland,2017,E-commerce activity (% of individuals buying o...,NaN,NaN,True,Usage and Ownership
130,Jamaica,2017,E-commerce activity (% of individuals buying o...,NaN,NaN,True,Usage and Ownership
131,Oman,2017,E-commerce activity (% of individuals buying o...,NaN,NaN,True,Usage and Ownership
132,Qatar,2017,E-commerce activity (% of individuals buying o...,NaN,NaN,True,Usage and Ownership


In [176]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [177]:
### 23. Mobile Device Penetration

In [178]:
indicators[22]

# load data
indicator = indicators[22]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))


Mobile Device Penetration
ITU_database


In [179]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [180]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [181]:
df = df[(df['Indicator name'] == 'Individuals owning a mobile phone (%)')]
df = df[(df.Year == 2019)]
df

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
5753,Angola,Africa,AGO,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5754,Benin,Africa,BEN,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5755,Botswana,Africa,BWA,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5756,Burkina Faso,Africa,BFA,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5757,Burundi,Africa,BDI,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5943,Suriname,The Americas,SUR,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5944,Trinidad and Tobago,The Americas,TTO,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5945,United States,The Americas,USA,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5946,Uruguay,The Americas,URY,Individuals owning a mobile phone (%),2019.0,83.282233,NaN,NaN


In [182]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [183]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5753,Angola,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and Ownership
5754,Benin,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and Ownership
5755,Botswana,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and Ownership
5756,Burkina Faso,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and Ownership
5757,Burundi,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and Ownership
...,...,...,...,...,...,...,...
5943,Suriname,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and Ownership
5944,Trinidad and Tobago,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and Ownership
5945,United States,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and Ownership
5946,Uruguay,2019.0,Mobile Device Penetration,83.282233,5.164112,True,Usage and Ownership


In [184]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [185]:
### 24. Mobile Device Penetration (female)

In [186]:
indicators[23]

# load data
indicator = indicators[23]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Mobile Device Penetration (female)
ITU_database


In [187]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [188]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [189]:
df = df[(df.Year==2019)]
df = df[(df['Indicator name'] == 'Female mobile phone ownership as a % of total female population')]

In [190]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [191]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
1763,Angola,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and Ownership
1764,Benin,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and Ownership
1765,Botswana,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and Ownership
1766,Burkina Faso,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and Ownership
1767,Burundi,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and Ownership
...,...,...,...,...,...,...,...
1953,Suriname,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and Ownership
1954,Trinidad and Tobago,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and Ownership
1955,United States,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and Ownership
1956,Uruguay,2019.0,Mobile Device Penetration (female),84.173624,5.208681,True,Usage and Ownership


In [192]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [193]:
### 25. Internet Usage

In [194]:
indicators[24]

# load data
indicator = indicators[24]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Internet Usage
ITU_database


In [195]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [196]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [197]:
df = df[(df['Indicator name'] == 'Individuals using the Internet, total (%)')]
df = df[(df.Year == 2019)]

In [198]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

In [199]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
47607,Cabo Verde,2019.0,Internet Usage,61.943398,4.097170,True,Usage and Ownership
47608,Central African Rep.,2019.0,Internet Usage,NaN,NaN,True,Usage and Ownership
47609,Congo (Rep. of the),2019.0,Internet Usage,NaN,NaN,True,Usage and Ownership
47610,Côte d'Ivoire,2019.0,Internet Usage,36.288955,2.814448,True,Usage and Ownership
47611,Equatorial Guinea,2019.0,Internet Usage,NaN,NaN,True,Usage and Ownership
...,...,...,...,...,...,...,...
47739,Saint Vincent and the Grenadines,2019.0,Internet Usage,NaN,NaN,True,Usage and Ownership
47740,Trinidad and Tobago,2019.0,Internet Usage,NaN,NaN,True,Usage and Ownership
47741,United States,2019.0,Internet Usage,89.430285,5.471514,True,Usage and Ownership
47742,Uruguay,2019.0,Internet Usage,83.351534,5.167577,True,Usage and Ownership


In [200]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [201]:
### 26. Households with a computer and with Internet Access

In [202]:
indicators[25]

# load data
indicator = indicators[25]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Households with a computer and with Internet access
ITU_database


In [203]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [204]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [205]:
df = df[(df['Indicator name'] == 'Households with Internet access at home (%)')]
df = df[(df.Year == 2020)]
df

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
45188,Mauritius,Africa,MUS,Households with Internet access at home (%),2020.0,72.642477,NaN,NaN
45189,Bahrain,Arab States,BHR,Households with Internet access at home (%),2020.0,99.671115,NaN,NaN
45190,Egypt,Arab States,EGY,Households with Internet access at home (%),2020.0,73.011528,NaN,NaN
45191,Kuwait,Arab States,KWT,Households with Internet access at home (%),2020.0,99.399945,NaN,NaN
45192,Morocco,Arab States,MAR,Households with Internet access at home (%),2020.0,84.520014,NaN,NaN
...,...,...,...,...,...,...,...,...
45250,Cuba,The Americas,CUB,Households with Internet access at home (%),2020.0,33.309090,NaN,NaN
45251,Ecuador,The Americas,ECU,Households with Internet access at home (%),2020.0,53.209495,NaN,NaN
45252,Mexico,The Americas,MEX,Households with Internet access at home (%),2020.0,60.553970,NaN,NaN
45253,Paraguay,The Americas,PRY,Households with Internet access at home (%),2020.0,36.533608,NaN,NaN


In [206]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [207]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
45188,Mauritius,2020.0,Households with a computer and with Internet a...,72.642477,4.632124,True,Usage and Ownership
45189,Bahrain,2020.0,Households with a computer and with Internet a...,99.671115,5.983556,True,Usage and Ownership
45190,Egypt,2020.0,Households with a computer and with Internet a...,73.011528,4.650576,True,Usage and Ownership
45191,Kuwait,2020.0,Households with a computer and with Internet a...,99.399945,5.969997,True,Usage and Ownership
45192,Morocco,2020.0,Households with a computer and with Internet a...,84.520014,5.226001,True,Usage and Ownership
...,...,...,...,...,...,...,...
45250,Cuba,2020.0,Households with a computer and with Internet a...,33.309090,2.665454,True,Usage and Ownership
45251,Ecuador,2020.0,Households with a computer and with Internet a...,53.209495,3.660475,True,Usage and Ownership
45252,Mexico,2020.0,Households with a computer and with Internet a...,60.553970,4.027699,True,Usage and Ownership
45253,Paraguay,2020.0,Households with a computer and with Internet a...,36.533608,2.826680,True,Usage and Ownership


In [208]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [209]:
### 27. % Social media penetration

In [210]:
indicators[26]

# load data
indicator = indicators[26]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Social media penetration
social_media_penetration


In [211]:
df.head(15)

,Country Name,Active social network penetration in selected countries and territories as of January 2021,Unnamed: 2
0,UAE,99.0,NaN
1,South Korea,89.3,NaN
2,Taiwan,88.1,NaN
3,Netherlands,88.0,NaN
4,Malaysia,86.0,NaN
5,Hong Kong,85.6,NaN
6,Canada,84.9,NaN
7,Singapore,84.4,NaN
8,Denmark,83.6,NaN
9,Sweden,82.1,NaN


In [212]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [213]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Active social network penetration in selected countries and territories as of January 2021'] 
df['Year'] = 2021
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [214]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,UAE,2021,Social media penetration,99.0,5.950,True,Usage and Ownership
1,South Korea,2021,Social media penetration,89.3,5.465,True,Usage and Ownership
2,Taiwan,2021,Social media penetration,88.1,5.405,True,Usage and Ownership
3,Netherlands,2021,Social media penetration,88.0,5.400,True,Usage and Ownership
4,Malaysia,2021,Social media penetration,86.0,5.300,True,Usage and Ownership
5,Hong Kong,2021,Social media penetration,85.6,5.280,True,Usage and Ownership
6,Canada,2021,Social media penetration,84.9,5.245,True,Usage and Ownership
7,Singapore,2021,Social media penetration,84.4,5.220,True,Usage and Ownership
8,Denmark,2021,Social media penetration,83.6,5.180,True,Usage and Ownership
9,Sweden,2021,Social media penetration,82.1,5.105,True,Usage and Ownership


In [215]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [216]:
### 28. % of population using Facebook

In [217]:
indicators[27]

# load data
indicator = indicators[27]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population using Facebook
FB_users


In [218]:
df.head(15)

,Country,Facebook Users,Population in Thousands (2021),Percentage of Facebook Users
0,India,"251,000,000","1,393,409",18.01
1,United States,"240,000,000","332,915",72.09
2,Brazil,"139,000,000","213,993",64.96
3,Indonesia,"136,960,000","276,362",49.56
4,Mexico,"78,000,000","130,262",59.88
5,Philippines,"71,760,000","111,047",64.62
6,Vietnam,"66,720,000","98,169",67.96
7,Thailand,"46,000,000","69,951",65.76
8,United Kingdom,"44,000,000","68,207",64.51
9,Turkey,"44,000,000","85,043",51.74


In [219]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [220]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Percentage of Facebook Users'] 
df['Year'] = 2021
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [221]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,India,2021,% of population using Facebook,18.01,1.9005,True,Usage and Ownership
1,United States,2021,% of population using Facebook,72.09,4.6045,True,Usage and Ownership
2,Brazil,2021,% of population using Facebook,64.96,4.2480,True,Usage and Ownership
3,Indonesia,2021,% of population using Facebook,49.56,3.4780,True,Usage and Ownership
4,Mexico,2021,% of population using Facebook,59.88,3.9940,True,Usage and Ownership
...,...,...,...,...,...,...,...
223,Tuvalu,2021,% of population using Facebook,14.17,1.7085,True,Usage and Ownership
224,Niue,2021,% of population using Facebook,41.00,3.0500,True,Usage and Ownership
225,Tokelau,2021,% of population using Facebook,41.00,3.0500,True,Usage and Ownership
226,Vatican City,2021,% of population using Facebook,2.00,1.1000,True,Usage and Ownership


In [222]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [223]:
### 29. Gender gap for social media use

In [224]:
indicators[28]

# load data
indicator = indicators[28]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Gender gap for social media use
gender_gaps


In [225]:
df.head(15)

,ISO Code,Country,Region,Year,Gender parity in account ownership,Gender gap in social media use,Gender gap in mobile ownership
0,AFG,Afghanistan,South Asia,2014,11.109999,0.000000,0.000000
1,AFG,Afghanistan,South Asia,2015,14.087173,0.000000,0.000000
2,AFG,Afghanistan,South Asia,2016,17.064350,0.000000,0.000000
3,AFG,Afghanistan,South Asia,2017,20.041523,0.000000,0.000000
4,AFG,Afghanistan,South Asia,2018,20.041523,0.000000,0.000000
5,AFG,Afghanistan,South Asia,2019,20.041523,0.000000,0.000000
6,AGO,Angola,Sub-Saharan Africa,2014,55.237572,46.428570,51.645042
7,AGO,Angola,Sub-Saharan Africa,2015,55.237572,46.428570,57.001461
8,AGO,Angola,Sub-Saharan Africa,2016,55.237572,46.428570,63.725491
9,AGO,Angola,Sub-Saharan Africa,2017,55.237572,38.646553,53.966476


In [226]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [227]:
df = df[df.Year==2019]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Gender gap in social media use'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [228]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,Gender gap for social media use,0.000000,1.000000,True,Usage and Ownership
11,Angola,2019,Gender gap for social media use,49.552494,3.477625,True,Usage and Ownership
17,Albania,2019,Gender gap for social media use,41.406120,3.070306,True,Usage and Ownership
23,United Arab Emirates,2019,Gender gap for social media use,13.890497,1.694525,True,Usage and Ownership
29,Argentina,2019,Gender gap for social media use,100.000000,6.000000,True,Usage and Ownership
...,...,...,...,...,...,...,...
995,Samoa,2019,Gender gap for social media use,100.000000,6.000000,True,Usage and Ownership
1001,Yemen,2019,Gender gap for social media use,0.000000,1.000000,True,Usage and Ownership
1007,South Africa,2019,Gender gap for social media use,96.334473,5.816724,True,Usage and Ownership
1013,Zambia,2019,Gender gap for social media use,59.793156,3.989658,True,Usage and Ownership


In [229]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [230]:
### 30. % of population using digital financial services

In [231]:
indicators[29]

# load data
indicator = indicators[29]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population using digital financial services
population_digital_financial_services


In [232]:
df.head(15)

# May have to convert the data to csv file as the original file is missing many important columns

,Year,ISO,Country Name,Region,Income Group,Account (% age 15+),"Account, male (% age 15+)","Account, in labor force (% age 15+)","Account, out of labor force (% age 15+)","Account, female (% age 15+)",...,"Mobile money account, female (% age 15+)","Mobile money account, young adults (% age 15-24)","Mobile money account, older adults (% age 25+)","Mobile money account, primary education or less (% age 15+)","Mobile money account, secondary education or less (% age 15+)","Mobile money account, income, poorest 40% (% age 15+)","Mobile money account, income, richest 60% (% age 15+)","Mobile money account, rural (% age 15+)",data_country,data_year
0,2011,AFG,Afghanistan,South Asia,Low income,9%,15%,15%,2%,3%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,AFG,Afghanistan,South Asia,Low income,10%,16%,15%,4%,4%,...,0%,0%,0%,0%,0%,0%,1%,0%,NaN,NaN
2,2017,AFG,Afghanistan,South Asia,Low income,15%,23%,25%,4%,7%,...,1%,0%,1%,0%,2%,0%,1%,1%,NaN,NaN
3,2011,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,39%,39%,46%,31%,39%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,29%,36%,36%,12%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,28%,34%,36%,15%,23%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2014,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,38%,43%,44%,29%,34%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,40%,42%,53%,27%,38%,...,2%,6%,1%,1%,4%,0%,4%,2%,NaN,NaN
8,2011,ARB,Arab world,NaN,NaN,22%,30%,33%,11%,14%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014,ARB,Arab world,NaN,NaN,30%,38%,42%,18%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [233]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [234]:
df = df[(df.Year == 2017)]

# Must convert the data to float by removing the % sign
df['Used the internet to pay bills or to buy something online in the past year (% age 15+)'] = df['Used the internet to pay bills or to buy something online in the past year (% age 15+)'].str.replace('%','')
df['Used the internet to pay bills or to buy something online in the past year (% age 15+)'] = df['Used the internet to pay bills or to buy something online in the past year (% age 15+)'].astype(float)

In [235]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Used the internet to pay bills or to buy something online in the past year (% age 15+)'] 
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [236]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df 

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
2,Afghanistan,2017,% of population using digital financial services,1.0,1.05,True,Usage and Ownership
7,Albania,2017,% of population using digital financial services,7.0,1.35,True,Usage and Ownership
10,Arab world,2017,% of population using digital financial services,9.0,1.45,True,Usage and Ownership
13,United Arab Emirates,2017,% of population using digital financial services,60.0,4.00,True,Usage and Ownership
16,Argentina,2017,% of population using digital financial services,19.0,1.95,True,Usage and Ownership
...,...,...,...,...,...,...,...
479,World,2017,% of population using digital financial services,29.0,2.45,True,Usage and Ownership
482,Kosovo,2017,% of population using digital financial services,15.0,1.75,True,Usage and Ownership
487,South Africa,2017,% of population using digital financial services,14.0,1.70,True,Usage and Ownership
490,Zambia,2017,% of population using digital financial services,11.0,1.55,True,Usage and Ownership


In [237]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [238]:
### 31. Mobile Broadband Pricing (pre-paid)

In [239]:
indicators[30]

# load data
indicator = indicators[30]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Mobile Broadband Pricing (pre-paid)
ITU_database


In [240]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [241]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [242]:
df = df[df.Year==2019]
df = df[df['Indicator name'] == 'Mobile broadband basket as a % of GNI p.c.']
df

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
17974,Angola,Africa,AGO,Mobile broadband basket as a % of GNI p.c.,2019.0,5.63,NaN,NaN
17975,Benin,Africa,BEN,Mobile broadband basket as a % of GNI p.c.,2019.0,6.21,NaN,NaN
17976,Botswana,Africa,BWA,Mobile broadband basket as a % of GNI p.c.,2019.0,2.87,NaN,NaN
17977,Burkina Faso,Africa,BFA,Mobile broadband basket as a % of GNI p.c.,2019.0,19.63,NaN,NaN
17978,Burundi,Africa,BDI,Mobile broadband basket as a % of GNI p.c.,2019.0,20.43,NaN,NaN
...,...,...,...,...,...,...,...,...
18161,Saint Vincent and the Grenadines,The Americas,VCT,Mobile broadband basket as a % of GNI p.c.,2019.0,4.59,NaN,NaN
18162,Suriname,The Americas,SUR,Mobile broadband basket as a % of GNI p.c.,2019.0,1.93,NaN,NaN
18163,Trinidad and Tobago,The Americas,TTO,Mobile broadband basket as a % of GNI p.c.,2019.0,3.07,NaN,NaN
18164,United States,The Americas,USA,Mobile broadband basket as a % of GNI p.c.,2019.0,0.42,NaN,NaN


In [243]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (6-row)+1)

In [244]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
17974,Angola,2019.0,Mobile Broadband Pricing (pre-paid),5.63,5.7185,True,Usage and Ownership
17975,Benin,2019.0,Mobile Broadband Pricing (pre-paid),6.21,5.6895,True,Usage and Ownership
17976,Botswana,2019.0,Mobile Broadband Pricing (pre-paid),2.87,5.8565,True,Usage and Ownership
17977,Burkina Faso,2019.0,Mobile Broadband Pricing (pre-paid),19.63,5.0185,True,Usage and Ownership
17978,Burundi,2019.0,Mobile Broadband Pricing (pre-paid),20.43,4.9785,True,Usage and Ownership
...,...,...,...,...,...,...,...
18161,Saint Vincent and the Grenadines,2019.0,Mobile Broadband Pricing (pre-paid),4.59,5.7705,True,Usage and Ownership
18162,Suriname,2019.0,Mobile Broadband Pricing (pre-paid),1.93,5.9035,True,Usage and Ownership
18163,Trinidad and Tobago,2019.0,Mobile Broadband Pricing (pre-paid),3.07,5.8465,True,Usage and Ownership
18164,United States,2019.0,Mobile Broadband Pricing (pre-paid),0.42,5.9790,True,Usage and Ownership


In [245]:
### 32. Tax as % of total cost of mobile ownership

In [246]:
indicators[31]

# load data
indicator = indicators[31]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Tax as % of total cost of mobile ownership
tax_percent_mobile_ownership


In [247]:
df.head(15)

,ISO Code,Country,Region,Year,Cluster,Index,Infrastructure,Affordability,Consumer Readiness,Content and Services,...,Gender gap in mobile ownership,TLDs per capita,E-Government Score,Mobile Social Media Penetration,Apps developed per person,Number of apps in national language,Accessibility of top ranked apps,Cybersecurity Index,data_country,data_year
0,AFG,Afghanistan,South Asia,2014,Discoverer,22.12,21.74,31.79,24.40,14.19,...,0.00,39.55,18.11,3.28,20.98,2.44,4.37,26.50,NaN,NaN
1,AFG,Afghanistan,South Asia,2015,Discoverer,22.99,22.82,30.81,25.28,15.71,...,0.00,39.57,24.27,4.36,22.93,2.79,8.03,25.83,NaN,NaN
2,AFG,Afghanistan,South Asia,2016,Discoverer,23.71,26.92,26.75,26.07,16.83,...,0.00,39.58,30.43,6.73,30.31,2.85,5.90,25.17,NaN,NaN
3,AFG,Afghanistan,South Asia,2017,Discoverer,25.82,33.54,27.22,28.56,17.04,...,0.00,39.47,30.50,7.78,31.62,2.91,6.15,24.50,NaN,NaN
4,AFG,Afghanistan,South Asia,2018,Discoverer,28.39,30.91,42.64,29.24,16.87,...,0.00,39.39,30.56,8.54,36.54,2.96,8.66,17.70,NaN,NaN
5,AFG,Afghanistan,South Asia,2019,Discoverer,28.94,32.34,41.53,29.72,17.58,...,0.00,39.41,41.18,9.39,39.36,3.00,5.72,17.70,NaN,NaN
6,AGO,Angola,Sub-Saharan Africa,2014,Discoverer,32.78,25.99,35.99,44.33,27.85,...,51.65,0.00,29.92,3.99,22.74,53.33,49.09,8.80,NaN,NaN
7,AGO,Angola,Sub-Saharan Africa,2015,Emerging,37.18,33.09,42.01,45.41,30.27,...,57.00,0.31,32.35,5.26,22.12,55.08,58.33,8.47,NaN,NaN
8,AGO,Angola,Sub-Saharan Africa,2016,Emerging,39.85,37.80,44.74,46.47,32.11,...,63.73,0.00,34.78,6.27,27.46,56.52,61.73,8.13,NaN,NaN
9,AGO,Angola,Sub-Saharan Africa,2017,Emerging,42.89,48.60,47.94,46.32,31.36,...,53.97,0.00,37.88,4.26,31.94,57.06,55.09,7.80,NaN,NaN


In [248]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [249]:
df = df[df.Year==2019]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
df['data_col'] = df['Tax as a % of TCMO'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (6-row)+1)

In [250]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,Tax as % of total cost of mobile ownership,79.16,2.0420,False,Usage and Ownership
11,Angola,2019,Tax as % of total cost of mobile ownership,65.83,2.7085,False,Usage and Ownership
17,Albania,2019,Tax as % of total cost of mobile ownership,50.00,3.5000,False,Usage and Ownership
23,United Arab Emirates,2019,Tax as % of total cost of mobile ownership,82.31,1.8845,False,Usage and Ownership
29,Argentina,2019,Tax as % of total cost of mobile ownership,5.23,5.7385,False,Usage and Ownership
...,...,...,...,...,...,...,...
995,Samoa,2019,Tax as % of total cost of mobile ownership,62.50,2.8750,False,Usage and Ownership
1001,Yemen,2019,Tax as % of total cost of mobile ownership,75.00,2.2500,False,Usage and Ownership
1007,South Africa,2019,Tax as % of total cost of mobile ownership,62.50,2.8750,False,Usage and Ownership
1013,Zambia,2019,Tax as % of total cost of mobile ownership,17.88,5.1060,False,Usage and Ownership


In [251]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [252]:
### 33. % of population with a smartphone

In [253]:
indicators[32]

# load data
indicator = indicators[32]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population with a smartphone
population_with_smartphones


In [254]:
df.head(15)

,Unnamed: 0,Country,Total Population,Smartphone Penetration Rate,Smartphone Users
0,1,China,1.44B,0.64,918.45M
1,2,India,1.38B,0.32,439.42M
2,3,United States,331M,0.82,270M
3,4,Indonesia,273.52M,0.59,160.23M
4,5,Brazil,212.56M,0.51,109.34M
5,6,Russia,145.93M,0.69,99.93M
6,7,Japan,126.48M,0.63,80M
7,8,Mexico,128.93M,0.54,70.14M
8,9,Germany,83.78M,0.78,65.24M
9,10,Vietnam,97.34M,0.63,61.37M


In [255]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [256]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Smartphone Penetration Rate'] 
df['Country Name'] = df['Country']
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_b(row,old_min=0,old_max=1))

In [257]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,China,2020,% of population with a smartphone,0.64,4.20,True,Usage and Ownership
1,India,2020,% of population with a smartphone,0.32,2.60,True,Usage and Ownership
2,United States,2020,% of population with a smartphone,0.82,5.10,True,Usage and Ownership
3,Indonesia,2020,% of population with a smartphone,0.59,3.95,True,Usage and Ownership
4,Brazil,2020,% of population with a smartphone,0.51,3.55,True,Usage and Ownership
5,Russia,2020,% of population with a smartphone,0.69,4.45,True,Usage and Ownership
6,Japan,2020,% of population with a smartphone,0.63,4.15,True,Usage and Ownership
7,Mexico,2020,% of population with a smartphone,0.54,3.70,True,Usage and Ownership
8,Germany,2020,% of population with a smartphone,0.78,4.90,True,Usage and Ownership
9,Vietnam,2020,% of population with a smartphone,0.63,4.15,True,Usage and Ownership


In [258]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [259]:
### 34. Gender gap in internet usage

In [260]:
indicators[33]

# load data
indicator = indicators[33]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Gender gap in internet usage
ITU_database


In [261]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [262]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [263]:
df = df[(df['Indicator name'] == 'Female Internet users as a % of total female population')]
df = df[(df.Year == 2019)]
df

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
40307,Cabo Verde,Africa,CPV,Female Internet users as a % of total female p...,2019.0,60.965017,NaN,NaN
40308,Côte d'Ivoire,Africa,CIV,Female Internet users as a % of total female p...,2019.0,32.924445,NaN,NaN
40309,Kenya,Africa,KEN,Female Internet users as a % of total female p...,2019.0,20.125122,NaN,NaN
40310,Lesotho,Africa,LSO,Female Internet users as a % of total female p...,2019.0,44.922768,NaN,NaN
40311,Mauritius,Africa,MUS,Female Internet users as a % of total female p...,2019.0,60.130645,NaN,NaN
...,...,...,...,...,...,...,...,...
40388,Mexico,The Americas,MEX,Female Internet users as a % of total female p...,2019.0,68.574653,NaN,NaN
40389,Panama,The Americas,PAN,Female Internet users as a % of total female p...,2019.0,63.811494,NaN,NaN
40390,Paraguay,The Americas,PRY,Female Internet users as a % of total female p...,2019.0,69.017230,NaN,NaN
40391,Peru,The Americas,PER,Female Internet users as a % of total female p...,2019.0,56.992144,NaN,NaN


In [264]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [265]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
40307,Cabo Verde,2019.0,Gender gap in internet usage,60.965017,4.048251,True,Usage and Ownership
40308,Côte d'Ivoire,2019.0,Gender gap in internet usage,32.924445,2.646222,True,Usage and Ownership
40309,Kenya,2019.0,Gender gap in internet usage,20.125122,2.006256,True,Usage and Ownership
40310,Lesotho,2019.0,Gender gap in internet usage,44.922768,3.246138,True,Usage and Ownership
40311,Mauritius,2019.0,Gender gap in internet usage,60.130645,4.006532,True,Usage and Ownership
...,...,...,...,...,...,...,...
40388,Mexico,2019.0,Gender gap in internet usage,68.574653,4.428733,True,Usage and Ownership
40389,Panama,2019.0,Gender gap in internet usage,63.811494,4.190575,True,Usage and Ownership
40390,Paraguay,2019.0,Gender gap in internet usage,69.017230,4.450861,True,Usage and Ownership
40391,Peru,2019.0,Gender gap in internet usage,56.992144,3.849607,True,Usage and Ownership


In [266]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [267]:
### 35. Gender gap in mobile usage

In [268]:
indicators[34]

# load data
indicator = indicators[34]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Gender gap in mobile usage
gender_gaps


In [269]:
df.head(15)

,ISO Code,Country,Region,Year,Gender parity in account ownership,Gender gap in social media use,Gender gap in mobile ownership
0,AFG,Afghanistan,South Asia,2014,11.109999,0.000000,0.000000
1,AFG,Afghanistan,South Asia,2015,14.087173,0.000000,0.000000
2,AFG,Afghanistan,South Asia,2016,17.064350,0.000000,0.000000
3,AFG,Afghanistan,South Asia,2017,20.041523,0.000000,0.000000
4,AFG,Afghanistan,South Asia,2018,20.041523,0.000000,0.000000
5,AFG,Afghanistan,South Asia,2019,20.041523,0.000000,0.000000
6,AGO,Angola,Sub-Saharan Africa,2014,55.237572,46.428570,51.645042
7,AGO,Angola,Sub-Saharan Africa,2015,55.237572,46.428570,57.001461
8,AGO,Angola,Sub-Saharan Africa,2016,55.237572,46.428570,63.725491
9,AGO,Angola,Sub-Saharan Africa,2017,55.237572,38.646553,53.966476


In [270]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and Ownership


In [271]:
df = df[(df.Year == 2019)]
df

,ISO Code,Country,Region,Year,Gender parity in account ownership,Gender gap in social media use,Gender gap in mobile ownership
5,AFG,Afghanistan,South Asia,2019,2.004152e+01,0.000000,0.000000
11,AGO,Angola,Sub-Saharan Africa,2019,5.523757e+01,49.552494,57.712311
17,ALB,Albania,Europe & Central Asia,2019,8.901575e+01,41.406120,67.741936
23,ARE,United Arab Emirates,Middle East & North Africa,2019,7.944289e+01,13.890497,100.000000
29,ARG,Argentina,Latin America & Caribbean,2019,1.000000e+02,100.000000,99.650528
...,...,...,...,...,...,...,...
995,WSM,Samoa,East Asia & Pacific,2019,9.256590e+01,100.000000,100.000000
1001,YEM,Yemen,Middle East & North Africa,2019,3.250000e-15,0.000000,0.000000
1007,ZAF,South Africa,Sub-Saharan Africa,2019,1.000000e+02,96.334473,78.272583
1013,ZMB,Zambia,Sub-Saharan Africa,2019,7.414376e+01,59.793156,53.703701


In [272]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Gender gap in mobile ownership'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [273]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,Gender gap in mobile usage,0.000000,1.000000,True,Usage and Ownership
11,Angola,2019,Gender gap in mobile usage,57.712311,3.885616,True,Usage and Ownership
17,Albania,2019,Gender gap in mobile usage,67.741936,4.387097,True,Usage and Ownership
23,United Arab Emirates,2019,Gender gap in mobile usage,100.000000,6.000000,True,Usage and Ownership
29,Argentina,2019,Gender gap in mobile usage,99.650528,5.982526,True,Usage and Ownership
...,...,...,...,...,...,...,...
995,Samoa,2019,Gender gap in mobile usage,100.000000,6.000000,True,Usage and Ownership
1001,Yemen,2019,Gender gap in mobile usage,0.000000,1.000000,True,Usage and Ownership
1007,South Africa,2019,Gender gap in mobile usage,78.272583,4.913629,True,Usage and Ownership
1013,Zambia,2019,Gender gap in mobile usage,53.703701,3.685185,True,Usage and Ownership


In [274]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [275]:
### Score Aggregating

In [276]:
import os

In [277]:
# get list of files in scores folder
scores = os.listdir('../indicator_scores/')
scores = [s for s in scores if s.startswith('people')]

In [278]:
scores

['people_% of internet users who own cryptocurrency_scores.csv',
 'people_% of population using digital financial services_scores.csv',
 'people_% of population using Facebook_scores.csv',
 'people_% of population using internet (all)_scores.csv',
 'people_% of population using internet (female)_scores.csv',
 'people_% of population using internet (male)_scores.csv',
 'people_% of population with a smartphone_scores.csv',
 'people_Automation-led unemployment_scores.csv',
 'people_Cyberbullying_scores.csv',
 'people_E-commerce activity (% of individuals buying online and frequency)_scores.csv',
 'people_E-waste generated, kilograms per inhabitant_scores.csv',
 'people_Financial Inclusiveness_scores.csv',
 'people_Gender gap for social media use_scores.csv',
 'people_Gender gap in internet usage_scores.csv',
 'people_Gender gap in mobile usage_scores.csv',
 'people_Global Crypto Adoption Index_scores.csv',
 'people_Global Wellbeing Initiative (World Happiness Index)_scores.csv',
 'people

In [279]:
# create a dataframe that concatenates all these file into one table
df = pd.concat([pd.read_csv('../indicator_scores/{}'.format(s)) for s in scores])    

In [280]:
df.to_csv('../pillar_scores/people_scores_beta.csv')

In [281]:
# Data cleaning
df['new_rank_score'] = df['new_rank_score'].fillna(0)
df.sort_values(by=['Country Name'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df['Country Name'] = df['Country Name'].astype(str, errors = 'ignore')

# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()
df['Country Name'] = df['Country Name'].str.strip('**')
df['Country Name'] = df['Country Name'].str.strip('*')
df['Country Name'] = df['Country Name'].str.strip(' *')

# Replace values that are not truly country names with nan
df['Country Name'] = df['Country Name'].replace('nan',np.nan)

df['Country Name'] = df['Country Name'].replace('Vietnam','Viet Nam')
df['Country Name'] = df['Country Name'].replace('United States of America','United States')



# Dropping the columns having NaN/NaT values
df = df[df['Country Name'].notna()]


In [282]:
sorted(df['Country Name'].unique().tolist())

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Arab world',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bolivia (Plurinational State of)',
 'Bolivia, Plurinational State of',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Rep.',
 'Central African Republic',
 'Chad',
 'Channel Islands',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo (Brazzaville)',
 'Congo (Democratic Republic of the)',
 'Congo (Rep. of the)',
 'Congo, Dem. Rep.',
 'Congo, Democratic Republic',
 'Congo, Rep.',
 'Congo, The Democratic Republic of

In [283]:
# average indicator scores per country
agg_df = df.groupby(['Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [284]:
agg_df.columns = ['agg_score', 'count_source' ]

In [285]:
max_number_sources = agg_df.describe()['count_source']['max']

In [286]:
agg_df['agg_score_wt'] = agg_df['agg_score']*(agg_df['count_source']/max_number_sources)

In [287]:
agg_df.sort_values(by='agg_score', ascending=False, inplace=True)

In [288]:
agg_df.head(25)

,agg_score,count_source,agg_score_wt
Country Name,,,
Korea (Rep. of),5.819488,8,1.551864
"Hong Kong, China (SAR)",5.745000,1,0.191500
"Hong Kong, China",5.656689,8,1.508450
United Kingdom of Great Britain and Northern Ireland,5.646000,1,0.188200
Korea (Republic of),5.580000,1,0.186000
Brunei,5.527000,1,0.184233
Republic of Korea,5.498500,1,0.183283
"Macao, China",5.375889,7,1.254374
Georgia (Country),5.358500,1,0.178617


In [289]:
agg_df.to_csv('../pillar_scores/people_scores_v0.csv')

In [290]:
### Score Aggregating by Subpillars

In [291]:
df.insert(0,'Pillar','People')
df

,Pillar,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar,Unnamed: 0
0,People,Afghanistan,2019.0,Mobile Device Penetration (female),NaN,0.000000,True,Usage and Ownership,NaN
1,People,Afghanistan,2019.0,Number of mobile apps available in national la...,3.0000,1.150000,True,Culture,NaN
2,People,Afghanistan,2019.0,UNDP Human Development Index (HDI),0.5110,3.555000,True,Usage and Ownership,173.0
3,People,Afghanistan,2020.0,Human Capital Index (HCI),0.3728,2.864000,True,Digital Literacy Skills,NaN
4,People,Afghanistan,2021.0,"E-waste generated, kilograms per inhabitant",0.5771,1.040883,True,Digital wellbeing,NaN
...,...,...,...,...,...,...,...,...,...
3766,People,Zimbabwe,2019.0,Tax as % of total cost of mobile ownership,24.0000,4.800000,False,Usage and Ownership,NaN
3767,People,Zimbabwe,2019.0,Number of mobile apps available in national la...,47.0000,3.350000,True,Culture,NaN
3768,People,Zimbabwe,2019.0,Global Crypto Adoption Index,0.0400,1.200000,True,Usage and Ownership,NaN
3769,People,Zimbabwe,2019.0,Mobile Device Penetration,NaN,0.000000,True,Usage and Ownership,NaN


In [292]:
sub_df = df.groupby(['Pillar','Sub-Pillar','Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [293]:
sub_df.columns = ['agg_score', 'count_source' ]

In [294]:
max_number_sources = sub_df.describe()['count_source']['max']

In [295]:
sub_df['agg_score_wt'] = sub_df['agg_score']*(sub_df['count_source']/max_number_sources)

In [296]:
sub_df.to_csv('../subpillar_score/people_scores_subpillar_v0.csv')

### Sources Generation

In [297]:
#Get all countries from Countries.xlsx
countries = pd.read_excel('../../data/Countries.xlsx')
col_names = ['Country or Area']
countries = countries[col_names]
countries.rename(columns = {'Country or Area': 'Country Name'}, inplace = True)

In [298]:
#Get all indicators from names dataframe retrieve at the begining of the script
bnames=bnames[['check','Sub-Pillar','Indicator','Data Source','Data Link']]
bnames.rename(columns = {'check': 'Pillar'}, inplace = True)

D:\Apps\Anaconda\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [299]:
#Do a nice cross join so that we have combination of all countries vs all indicators
sources = countries.merge(bnames, how='cross')
sources

,Country Name,Pillar,Sub-Pillar,Indicator,Data Source,Data Link
0,Algeria,People,Digital Literacy Skills,Human Capital Index (HCI),UN: E-Government Survey,https://publicadministration.un.org/egovkb
1,Algeria,People,Digital Literacy Skills,% of population using internet (all),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics
2,Algeria,People,Digital Literacy Skills,% of population using internet (female),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics
3,Algeria,People,Digital Literacy Skills,% of population using internet (male),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics
4,Algeria,People,Digital Literacy Skills,% of internet users who own cryptocurrency,Statista,https://www.statista.com
...,...,...,...,...,...,...
8710,Wallis and Futuna Islands,People,Usage and Ownership,Mobile Broadband Pricing (pre-paid),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics
8711,Wallis and Futuna Islands,People,Usage and Ownership,Tax as % of total cost of mobile ownership,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com
8712,Wallis and Futuna Islands,People,Usage and Ownership,% of population with a smartphone,Statista,https://www.statista.com
8713,Wallis and Futuna Islands,People,Usage and Ownership,Gender gap in internet usage,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics


In [300]:
#Make copy of a scores dataframe and add the column available, with value of 1 (string)
#denoting all the country/indicator combinations that have value
dfsources = df[['Country Name','Pillar','Sub-Pillar','Indicator']].copy()
dfsources['Available'] = '1'

In [301]:
#Merge (left join) sources (all country indicator combinations)
#with those in copied scores dataframe
#resulting in a sources dataframe with values of 1 in Available column where there is a value for country/indicator
#and there is a NaN where there's not. 
#subsequently replace NaN with 0 (string)
sources = sources.merge(dfsources, how='left', on=['Country Name','Pillar','Sub-Pillar','Indicator'])
sources['Available'] = sources['Available'].fillna('0')
sources

,Country Name,Pillar,Sub-Pillar,Indicator,Data Source,Data Link,Available
0,Algeria,People,Digital Literacy Skills,Human Capital Index (HCI),UN: E-Government Survey,https://publicadministration.un.org/egovkb,1
1,Algeria,People,Digital Literacy Skills,% of population using internet (all),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0
2,Algeria,People,Digital Literacy Skills,% of population using internet (female),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0
3,Algeria,People,Digital Literacy Skills,% of population using internet (male),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0
4,Algeria,People,Digital Literacy Skills,% of internet users who own cryptocurrency,Statista,https://www.statista.com,0
...,...,...,...,...,...,...,...
8710,Wallis and Futuna Islands,People,Usage and Ownership,Mobile Broadband Pricing (pre-paid),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0
8711,Wallis and Futuna Islands,People,Usage and Ownership,Tax as % of total cost of mobile ownership,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com,0
8712,Wallis and Futuna Islands,People,Usage and Ownership,% of population with a smartphone,Statista,https://www.statista.com,0
8713,Wallis and Futuna Islands,People,Usage and Ownership,Gender gap in internet usage,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0


In [302]:
#If sources.csv exists, get the contents, remove everyhing from this pillar, append prepared sources, save csv.
#if sources.csv does not exist, create new file from sources.
from os.path import exists

if exists('../../dashboard/Sources.csv') :
    CurrentSources = pd.read_csv('../../dashboard/Sources.csv', dtype=str)
    CurrentSources = CurrentSources[['Country Name','Pillar','Sub-Pillar','Indicator','Data Source','Data Link','Available']]
    CurrentSources = CurrentSources.loc[CurrentSources['Pillar'] != 'People']
    CurrentSources = CurrentSources.append(sources)
else :
    CurrentSources = sources
CurrentSources

,Country Name,Pillar,Sub-Pillar,Indicator,Data Source,Data Link,Available
0,Algeria,Regulation,Data standards and protection,data protection laws,UNCTAD: Data Protection and Privacy Legislatio...,https://unctad.org/page/data-protection-and-pr...,0
1,Algeria,Regulation,Data standards and protection,Global Tables of Data Privacy Laws and Bills,UNCTAD: Data Protection and Privacy Legislatio...,https://unctad.org/page/data-protection-and-pr...,0
2,Algeria,Regulation,Data standards and protection,Data Protection and Privacy Legislation Worldwide,UNCTAD: Data Protection and Privacy Legislatio...,https://unctad.org/page/data-protection-and-pr...,0
3,Algeria,Regulation,Cybersecurity,# key internet controls employed,Freedom House: Freedom on the Net,https://freedomhouse.org/report/freedom-net,0
4,Algeria,Regulation,Ethical Standards,Rule of Law Index,World Justice Project: Rule of Law Index,https://worldjusticeproject.org/our-work/resea...,1
...,...,...,...,...,...,...,...
8710,Wallis and Futuna Islands,People,Usage and Ownership,Mobile Broadband Pricing (pre-paid),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0
8711,Wallis and Futuna Islands,People,Usage and Ownership,Tax as % of total cost of mobile ownership,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com,0
8712,Wallis and Futuna Islands,People,Usage and Ownership,% of population with a smartphone,Statista,https://www.statista.com,0
8713,Wallis and Futuna Islands,People,Usage and Ownership,Gender gap in internet usage,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0


In [303]:
CurrentSources.to_csv('../../dashboard/Sources.csv', index=False)